In [26]:
import os
import glob
import dash
import datetime
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import dash_core_components as dcc
import dash_html_components as html
from collections import OrderedDict

## Auto Summary 

In [27]:
results = {}

# Build Results(One or more) and Baseline Results(One)

In [28]:
pwd= os.getcwd()
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'BW_Sustained_Multi-Threaded_Sequential_Read_Tests_by_Block_Size.html',
 'Chart_Gen.py',
 'Dashboard.py',
 'Executive_Summary.json',
 'ks.png',
 'ks_logo.jpg',
 'NVMef Performance QA Regression Reporting-N_Builds-Regression-Calculation.ipynb',
 'Post_Test_Data_Collecting_Cleaning .ipynb',
 'Regression_Cal.py',
 'Results_baseline_abstract_3.6.10307_2018-05-07_12-30-47',
 'Results_build_abstract_3.7.10408_2018-05-08_11-01-54',
 'Results_build_abstract_3.7.10435_2018-05-10_17-50-42',
 '__pycache__']

In [29]:
result_folder_build_1 = os.path.join(pwd, "Results_build_abstract_3.7.10408_2018-05-08_11-01-54")
result_folder_build_2 = os.path.join(pwd, "Results_build_abstract_3.7.10435_2018-05-10_17-50-42")
result_folder_build_3 = os.path.join(pwd, "Results_baseline_abstract_3.6.10307_2018-05-07_12-30-47")    # Baseline


# Like this we can have result_folder_build_3, result_folder_build_4, result_folder_build_n

# Loading the DataFrame for Baseline and Build

In [30]:
def results_df(*argv):

    runs_dict = {}
    num = 1
    for f in argv:
        f_name = os.path.basename(f)
        _ = f_name.lower().split("_")
        if "baseline" in _:
            result_file = os.path.join(f, "final.csv")
            df = pd.read_csv(result_file)
            runs_dict["baseline"] = [_[3], df]

          
        else:
            k = "build_" + str(num)
            result_file = os.path.join(f, "final.csv")
            df = pd.read_csv(result_file)
            runs_dict[k] = [_[3], df]
            num +=1
        
    
    return runs_dict
        
    

In [31]:
#all_runs = results_df(result_folder_baseline, result_folder_build_1, result_folder_build_2)
all_runs = results_df(result_folder_build_1, result_folder_build_2, result_folder_build_3)


## Version Checker


In [32]:
def version_check(all_tests):
    versions = []
    for i in all_runs:
        versions.append(all_runs[i][0])    
     
    return min(versions), max(versions)

# Bandwidth Charts

In [79]:
def bandwidth_blocksize(test_name, chart_title, all_tests):

    _, latest_version = version_check(all_tests)

    data = []
    bs_str = {  512: "512",
                1024: "1K",
                2048: "2K",
                4096: "4K",
                8192: "8K",
                16384: "16K",
                32768: "32K",
                65536: "64K",
                131072: "128K"
             }

    for test in all_tests:
        num = 1
        if test == "baseline":
            df_1 = all_tests[test][1]
            df_1_version = all_tests[test][0] + "(Baseline)"
            baseline_ver = df_1_version

            df_1 = df_1[df_1["Test_Name"] == test_name]
            df_1.index = range(1, len(df_1.index)+1)

            df_baseline_full = df_1.copy()

            df_1 = df_1[["Block Size", "Bandwidth (MB/s)", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]
            df_1["Block Size"] = [ bs_str[i] for i in df_1["Block Size"]]

            # Adding Data
            x_block_size = [str(i) for i in df_1["Block Size"]]

            y1_df1_BW = df_1["Bandwidth (MB/s)"]
            y2_df1_CPU = df_1["Appliance_CPU_Utilization"]
            y3_df1_Mem = df_1["Appliance_Memory_Utilization"]


            trace0=go.Scatter(x=x_block_size,
                                y = y1_df1_BW,
                                name = df_1_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_block_size,
                                y = y2_df1_CPU,
                                name = "CPU Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_block_size,
                                y = y3_df1_Mem,
                                name = "Memory Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)

            base_df = df_1.copy()

        else:

            df_run = all_tests[test][1]
            df_run_version = all_tests[test][0]

            df_run = df_run[df_run["Test_Name"] == test_name]
            df_run.index = range(1, len(df_run.index)+1)

            df_run_full = df_run.copy()

            df_run = df_run[["Block Size", "Bandwidth (MB/s)", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]
            df_run["Block Size"] = [ bs_str[i] for i in df_run["Block Size"]]

            # Adding Data
            x_block_size = [str(i) for i in df_run["Block Size"]]

            y1_df_run_BW = df_run["Bandwidth (MB/s)"]
            y2_df_run_CPU = df_run["Appliance_CPU_Utilization"]
            y3_df_run_Mem = df_run["Appliance_Memory_Utilization"]


            if df_run_version == latest_version:
                df_run_version += "(Latest)"
                latest_df = df_run.copy()
                latest_ver = df_run_version

            trace0 = go.Scatter(x = x_block_size,
                                y = y1_df_run_BW,
                                name = df_run_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_block_size,
                                y = y2_df_run_CPU,
                                name = "CPU Utilization" + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_block_size,
                                y = y3_df_run_Mem,
                                name = "Memory Utilization" + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)


    layout = go.Layout(title = chart_title,
                       xaxis = dict(title = "Block Size (in Bytes)"),
                       yaxis = dict(title = "Throughput (in MBps)" ),
                       yaxis2=dict(title='Appliance Resource Utilization (in %)',
                        titlefont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        tickfont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        overlaying='y',
                        side='right'), 
                      legend=dict(y=-.2, orientation='h'))


    fig = dict(data=data, layout=layout)

    f_name = "BW_" + chart_title.replace(" ", "_") + ".html"

    data = [pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df, df_baseline_full, df_run_full, baseline_ver, latest_ver, fig]

    #return pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df
    return data


## Chart 1 - Sustained Multi-Threaded Sequential Read Tests by Block Size


In [80]:
test_name = "Sustained Multi-Threaded Sequential Read Tests by Block Size"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version,  fig = bandwidth_blocksize(test_name = test_name,
                                                                                             chart_title="Sustained Multi-Threaded Sequential Read Tests by Block Size",
                                                                                             all_tests=all_runs)
test_name = test_name + "_BW"

results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]


## Chart 2 - Sustained Multi-Threaded Random Read Tests by Block Size(QD=256)


In [ ]:
test_name = "Sustained Multi-Threaded Random Read Tests by Block Size"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = bandwidth_blocksize(test_name = test_name,
                                                                                             chart_title="Sustained Multi-Threaded Random Read Tests by Block Size",
                                                                                             all_tests=all_runs)
test_name = test_name + "_BW"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]


## Chart 3 - Sequential Write Tests with Queue Depth=1 by Block Size


In [ ]:
test_name = "Sequential Write Tests with Queue Depth=1 by Block Size"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = bandwidth_blocksize(test_name = test_name,
                                                                                             chart_title="Sequential Write Tests with Queue Depth 1 by Block Size",
                                                                                             all_tests=all_runs)

test_name = test_name + "_BW"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]


# IOPS Charts

In [ ]:
def iops_qd(test_name, chart_title, all_tests):

    _, latest_version = version_check(all_tests)

    data =[]
    for test in all_tests:
        num = 1
        if test == "baseline":
            df_1 = all_tests[test][1]
            df_1_version = all_tests[test][0] + "(Baseline)"
            baseline_ver = df_1_version


            df_1 = df_1[df_1["Test_Name"] == test_name]
            df_1.index = range(1, len(df_1.index)+1)

            df_baseline_full = df_1.copy()

            df_1 = df_1[["Threads", "IOPS", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]


            # Adding Data
            x_qd = [str(i) for i in df_1["Threads"]]
            y1_df1_IOPS = df_1["IOPS"]
            y2_df1_CPU = df_1["Appliance_CPU_Utilization"]
            y3_df1_Mem = df_1["Appliance_Memory_Utilization"]


            trace0 = go.Scatter(x = x_qd,
                                y = y1_df1_IOPS,
                                name = df_1_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_qd,
                                y = y2_df1_CPU,
                                name = "CPU Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_qd,
                                y = y3_df1_Mem,
                                name = "Memory Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)

            base_df = df_1.copy()

        else:

            df_run = all_tests[test][1]
            df_run_version = all_tests[test][0]


            df_run = df_run[df_run["Test_Name"] == test_name]
            df_run.index = range(1, len(df_run.index)+1)

            df_run_full = df_run.copy()

            df_run = df_run[["Threads", "IOPS", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]


            # Adding Data
            x_qd = [str(i) for i in df_run["Threads"]]

            y1_df_run_IOPS = df_run["IOPS"]
            y2_df_run_CPU = df_run["Appliance_CPU_Utilization"]
            y3_df_run_Mem = df_run["Appliance_Memory_Utilization"]

            if df_run_version == latest_version:
                df_run_version += "(Latest)"
                latest_df = df_run.copy()
                latest_ver = df_run_version

            trace0 = go.Scatter(x = x_qd,
                                y = y1_df_run_IOPS,
                                name = df_run_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_qd,
                                y = y2_df_run_CPU,
                                name = "CPU Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_qd,
                                y = y3_df_run_Mem,
                                name = "Memory Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)


    layout = go.Layout(title = chart_title,
                       xaxis = dict(title = "Queue Depth"),
                       yaxis = dict(title = "IOPS" ),
                       yaxis2=dict(title='Appliance Resource Utilization (in %)',
                        titlefont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        tickfont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        overlaying='y',
                        side='right'
                    ))


    fig = dict(data=data, layout=layout)

    f_name = "IOPS_" + chart_title.replace(" ", "_") + ".html"

    data = [pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df, df_baseline_full, df_run_full, baseline_ver, latest_ver, fig]

    #return pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df
    return data


In [ ]:
def iops_bs(test_name, chart_title, all_tests):

    _, latest_version = version_check(all_tests)

    data = []
    bs_str = {  512: "512",
                1024: "1K",
                2048: "2K",
                4096: "4K",
                8192: "8K",
                16384: "16K",
                32768: "32K",
                65536: "64K",
                131072: "128K"
             }


    for test in all_tests:
        num = 1
        if test == "baseline":
            df_1 = all_tests[test][1]
            df_1_version = all_tests[test][0] + "(Baseline)"
            baseline_ver = df_1_version

            df_1 = df_1[df_1["Test_Name"] == test_name]
            df_1.index = range(1, len(df_1.index)+1)

            df_baseline_full = df_1.copy()

            df_1 = df_1[["Block Size", "IOPS", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]
            df_1["Block Size"] = [ bs_str[i] for i in df_1["Block Size"]]


            # Adding Data
            x_block_size = [str(i) for i in df_1["Block Size"]]
            y1_df1_IOPS = df_1["IOPS"]
            y2_df1_CPU = df_1["Appliance_CPU_Utilization"]
            y3_df1_Mem = df_1["Appliance_Memory_Utilization"]


            trace0 = go.Scatter(x = x_block_size,
                                y = y1_df1_IOPS,
                                name = df_1_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_block_size,
                                y = y2_df1_CPU,
                                name = "CPU Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_block_size,
                                y = y3_df1_Mem,
                                name = "Memory Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)

            base_df = df_1.copy()

        else:

            df_run = all_tests[test][1]
            df_run_version = all_tests[test][0]


            df_run = df_run[df_run["Test_Name"] == test_name]
            df_run.index = range(1, len(df_run.index)+1)

            df_run_full = df_run.copy()

            df_run = df_run[["Block Size", "IOPS", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization"]]
            df_run["Block Size"] = [ bs_str[i] for i in df_run["Block Size"]]

            # Adding Data
            x_block_size = [str(i) for i in df_run["Block Size"]]

            y1_df_run_IOPS = df_run["IOPS"]
            y2_df_run_CPU = df_run["Appliance_CPU_Utilization"]
            y3_df_run_Mem = df_run["Appliance_Memory_Utilization"]

            if df_run_version == latest_version:
                df_run_version += "(Latest)"
                latest_df = df_run.copy()
                latest_ver = df_run_version

            trace0 = go.Scatter(x = x_block_size,
                                y = y1_df_run_IOPS,
                                name = df_run_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_block_size,
                                y = y2_df_run_CPU,
                                name = "CPU Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_block_size,
                                y = y3_df_run_Mem,
                                name = "Memory Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)

    layout = go.Layout(title = chart_title,
                       xaxis = dict(title = "Block Size (in Bytes)"),
                       yaxis = dict(title = "IOPS" ),
                       yaxis2=dict(title='Appliance Resource Utilization (in %)',
                        titlefont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        tickfont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        overlaying='y',
                        side='right'
                    ))


    fig = dict(data=data, layout=layout)

    f_name = "IOPS_" + chart_title.replace(" ", "_") + ".html"

    data = [pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df, df_baseline_full, df_run_full, baseline_ver, latest_ver, fig]

    #return pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df
    return data

## Chart 4 - Sustained Multi-Threaded Random Read Tests by Block Size



In [ ]:
test_name = "Sustained Multi-Threaded Random Read Tests by Block Size"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = iops_bs(test_name = test_name,
                                                                                 chart_title="Sustained Multi-Threaded Random Read Tests by Block Size",
                                                                                 all_tests=all_runs)
test_name = test_name + "_IOPS"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]
a = results[test_name]

In [ ]:
a[1].head(2)

In [ ]:
a[2].head(2)

## Chart 5 - Sustained 4KB Random mixed 50% Write Tests by Threads



In [ ]:
test_name = "Sustained 4KB Random mixed 50% Write Tests by Threads"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = iops_qd(test_name = test_name,
                                                                                 chart_title="Sustained Random 4K Mixed Read 50 percent",
                                                                                 all_tests=all_runs)
test_name = test_name + "_IOPS"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]
a = results[test_name]

## Chart 6 - Sustained 4K Random Read



In [ ]:
test_name = "Sustained 4KB Random Read Tests by Number of Threads"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = iops_qd(test_name = test_name,
                                                                                 chart_title="Sustained 4K Random Read",
                                                                                 all_tests=all_runs)
test_name = test_name + "_IOPS"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]

## Chart 7 - Sustained 4K Random Write


In [ ]:
test_name = "Sustained 4KB Random Write Tests by Number of Threads"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = iops_qd(test_name = test_name,
                                                                                 chart_title="Sustained 4K Random Write",
                                                                                 all_tests=all_runs)
test_name = test_name + "_IOPS"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]

# Latency Charts

In [ ]:
def lat_qd(test_name, chart_title, all_tests):

    _, latest_version = version_check(all_tests)

    data =[]
    for test in all_tests:
        num = 1
        if test == "baseline":
            df_1 = all_tests[test][1]
            df_1_version = all_tests[test][0] + "(Baseline)"
            baseline_ver = df_1_version


            if "write" in test_name.lower():
                col = "Write Latency (us)"
            else:
                col = "Read Latency (us)"

            df_1 = df_1[df_1["Test_Name"] == test_name]
            df_1.index = range(1, len(df_1.index)+1)

            df_baseline_full = df_1.copy()

            df_1 = df_1[["Threads", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization", col]]


            # Adding Data
            x_qd = [str(i) for i in df_1["Threads"]]
            y1_df1_LAT = df_1[col]
            y2_df1_CPU = df_1["Appliance_CPU_Utilization"]
            y3_df1_Mem = df_1["Appliance_Memory_Utilization"]


            trace0 = go.Scatter(x = x_qd,
                                y = y1_df1_LAT,
                                name = df_1_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_qd,
                                y = y2_df1_CPU,
                                name = "CPU Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_qd,
                                y = y3_df1_Mem,
                                name = "Memory Utilization " + df_1_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)

            base_df = df_1.copy()

        else:

            df_run = all_tests[test][1]
            df_run_version = all_tests[test][0]

            if "write" in test_name.lower():
                col = "Write Latency (us)"
            else:
                col = "Read Latency (us)"

            df_run = df_run[df_run["Test_Name"] == test_name]
            df_run.index = range(1, len(df_run.index)+1)

            df_run_full = df_run.copy()

            df_run = df_run[["Threads", "IOPS", "Appliance_CPU_Utilization", "Appliance_Memory_Utilization", col]]


            # Adding Data
            x_qd = [str(i) for i in df_run["Threads"]]

            y1_df_run_LAT = df_run[col]
            y2_df_run_CPU = df_run["Appliance_CPU_Utilization"]
            y3_df_run_Mem = df_run["Appliance_Memory_Utilization"]

            if df_run_version == latest_version:
                df_run_version += "(Latest)"
                latest_df = df_run.copy()
                latest_ver = df_run_version

            trace0 = go.Scatter(x = x_qd,
                                y = y1_df_run_LAT,
                                name = df_run_version,
                                line = dict(width = 4),
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False)

            trace1 = go.Scatter(x = x_qd,
                                y = y2_df_run_CPU,
                                name = "CPU Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            trace2 = go.Scatter(x = x_qd,
                                y = y3_df_run_Mem,
                                name = "Memory Utilization " + df_run_version,
                                mode='markers',
                                hoverlabel = { "namelength" : -1},
                                cliponaxis = False,
                                yaxis='y2')

            data.append(trace0)
            data.append(trace1)
            data.append(trace2)


    layout = go.Layout(title = chart_title,
                       xaxis = dict(title = "Queue Depth"),
                       yaxis = dict(title = col ),
                       yaxis2=dict(title='Appliance Resource Utilization (in %)',
                        titlefont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        tickfont=dict(
                            color='rgb(148, 103, 189)'
                        ),
                        overlaying='y',
                        side='right'
                    ))


    fig = dict(data=data, layout=layout)

    f_name = "LAT_" + chart_title.replace(" ", "_") + ".html"

    data = [pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df, df_baseline_full, df_run_full, baseline_ver, latest_ver, fig]

    #return pyo.plot(fig, filename=f_name, auto_open=False), base_df, latest_df
    return data

## Chart 8 - Latency Test Sustained 4KB Random Read Tests

In [ ]:
test_name = "Latency Test Sustained 4KB Random Read Tests by Number of Threads"
chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = lat_qd(test_name = test_name,
                                                                                chart_title="Latency Test Sustained 4KB Random Read Tests by Number of Threads",
                                                                                all_tests=all_runs)

test_name = test_name + "_LAT"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]


## Chart 9 - Latency Test Sustained 4KB Random Read Tests

In [ ]:
test_name = "Latency Test Sustained 4KB Random Write Tests by Number of Threads"

chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig = lat_qd(test_name = test_name,
                                                                                chart_title="Latency Test Sustained 4KB Random Write Tests by Number of Threads",
                                                                                all_tests=all_runs)
test_name = test_name + "_LAT"
results[test_name] = [chart_path, base_df, latest_df, df_baseline_full, df_run_full, baseline_version, latest_version, fig]


# Regression Calculator

In [ ]:
regression_percent = -10
exe_summary = OrderedDict({"Bandwidth": [],
                           "Latency": [],
                           "IOPS": [],
                           "Latest Version": "",
                           "Baseline Version": "",
                           "Date": ""})


In [ ]:
def bw_reg_chk(test_name, df_base, df_current, base_version, latest_version):
    
    global exe_summary, regression_percent
    
    df = df_current.copy()
    df["Regression_Percent"] = (df_current["Bandwidth (MB/s)"] - df_base["Bandwidth (MB/s)"]) / df_base["Bandwidth (MB/s)"] * 100
    
    df_regression = df[df["Regression_Percent"] < regression_percent]

    if len(df_regression.index) == 0:
        pass 
    
    else:
        for index, row in df_regression.iterrows():
            bs = row["Block Size"]
            reg_p = int(row["Regression_Percent"])
            msg = "{}% regression is observed for the test '{}' for the Block_Size of {}".format(abs(reg_p), test_name,bs)
            exe_summary["Bandwidth"].append(msg)
        
    return df_regression


    


In [ ]:
def iops_qd_reg_chk(test_name, df_base, df_current, base_version, latest_version):
    
    global exe_summary, regression_percent
    
    df = df_current.copy()
    df["Regression_Percent"] = (df_current["IOPS"] - df_base["IOPS"]) / df_base["IOPS"] * 100
    
    df_regression = df[df["Regression_Percent"] < regression_percent]

    if len(df_regression.index) == 0:
        #exe_summary["IOPS"].append("No regression for the test '{}'".format(test_name))
        pass 
    
    else:
        for index, row in df_regression.iterrows():
            thread = row["Threads"]
            reg_p = int(row["Regression_Percent"])
            msg = "{}% regression is observed for the test '{}' with '{}' Threads ".format(abs(reg_p), test_name,thread)
            exe_summary["IOPS"].append(msg)
        
    return df_regression

In [ ]:
def iops_bs_reg_chk(test_name, df_base, df_current, base_version, latest_version):
    
    global exe_summary, regression_percent, bs_str
    
    df = df_current.copy()
    df["Regression_Percent"] = (df_current["IOPS"] - df_base["IOPS"]) / df_base["IOPS"] * 100
    
    df_regression = df[df["Regression_Percent"] < regression_percent]

    if len(df_regression.index) == 0:
        #exe_summary["IOPS"].append("No regression for the test '{}'".format(test_name))
        pass 
        
    else:
        for index, row in df_regression.iterrows():
            bs = row["Block Size"]
            reg_p = int(row["Regression_Percent"])
            msg = "{}% regression is observed for the test '{}' for the Block_Size of {}".format(abs(reg_p), test_name,bs)
            exe_summary["IOPS"].append(msg)
        
    return df_regression
    

In [ ]:
def lat_reg_chk(test_name, df_base, df_current, base_version, latest_version):
    
    global exe_summary, regression_percent
    
    if "write" in test_name.lower():
        col = "Write Latency (us)"
    else:
        col = "Read Latency (us)"
    
    df = df_current.copy()
    df["Regression_Percent"] = (df_current[col] - df_base[col]) / df_base[col] * 100
    
    df_regression = df[df["Regression_Percent"] < regression_percent]

    if len(df_regression.index) == 0:
        #exe_summary["Latency"].append("No regression for the test '{}'".format(test_name))
        pass 
        
    else:
        for index, row in df_regression.iterrows():
            threads = row["Threads"]
            reg_p = int(row["Regression_Percent"])
            msg = "{}% regression is observed for the test '{}' with '{}' Threads ".format(abs(reg_p), test_name,threads)
            exe_summary["Latency"].append(msg)
        
    return df_regression

In [ ]:
results.keys()

In [ ]:
from collections import OrderedDict

# 0 : Chart path
# 1 : baseline_df
# 2 : latest_df
# 3 : full_baseline_df
# 4 : full_latest_df
# 5 : baseline_version
# 6 : latest_version 
# 7 : plotly_fig object


def gen_summary(results):
    global exe_summary

    for data in results:
        if data.endswith("_BW"):
            test_name = data.split("_")[0]
            arg = test_name, results[data][1], results[data][2], results[data][5], results[data][6]
            bw_reg_chk(*arg)

        elif data.endswith("_IOPS"):
            test_name = data.split("_")[0]
            arg = test_name, results[data][1], results[data][2], results[data][5], results[data][6]
            if "block size" in data.lower():
                iops_bs_reg_chk(*arg)
            else:
                iops_qd_reg_chk(*arg)

        else:
            test_name = data.split("_")[0]
            arg = test_name, results[data][1], results[data][2], results[data][5], results[data][6]
            lat_reg_chk(*arg)

        exe_summary["Latest Version"] = arg[-1]
        exe_summary["Baseline Version"] = arg[-2]
        now = datetime.datetime.now()
        now_str = now.strftime("%Y-%m-%d %H:%M")
        exe_summary["Date"] = now_str

    summary = exe_summary

    return summary


In [ ]:
exe_summary

In [ ]:
summary = gen_summary(results)

In [ ]:
summary

# Dashboard

In [ ]:
import argparse
import json
import os
import dash_core_components as dcc 
import dash_html_components as html 
from collections import OrderedDict
from pprint import pprint as pp


import Chart_Gen
import Regression_Cal

In [ ]:
all_data = results

In [ ]:
app = dash.Dash()

In [ ]:
graph_fig = { data: all_data[data][-1] for data in all_data }


In [ ]:
app.layout = html.Div([ dcc.Graph(id=data, figure=graph_fig[data])   for data in graph_fig])


In [ ]:
type(app.layout)

In [ ]:
for i in results:
    print(i, results[i][-1].keys())

In [ ]:
graph_fig = { data: results[data][-1] for data in results}


In [ ]:
for i in graph_fig:
    print(i, graph_fig[i].keys())

In [ ]:
img_path = os.path.join(os.getcwd(), "ks_logo.jpg")

In [ ]:
img_path

In [ ]:
b = json.loads(a)

In [ ]:
a = json.dumps(exe_summary)

In [ ]:
exe_summary

In [ ]:
for d in exe_summary:
    print(exe_summary[d])

In [ ]:
df_summary = pd.DataFrame(columns=["Segement", "Summary"])

In [ ]:
df_summary[[1]] = [23, 23]

In [ ]:
exe_summary['Bandwidth']

In [ ]:
df_bw = pd.DataFrame(data=exe_summary['Bandwidth'], columns=["Bandwidth"])

In [ ]:
df_bw.index = range(1, len(df_bw.index)+1)

In [ ]:
df_bw

In [ ]:
def exe_summary_df(exe_summary):

    all_df = []

    for data in exe_summary:
        if data == "Bandwidth" and len(exe_summary[data]) != 0:
            df_bw = pd.DataFrame(data=exe_summary['Bandwidth'], columns=["Bandwidth"])
            df_bw.index = range(1, len(df_bw.index)+1)

            all_df.append(df_bw)

        elif data == "Latency" and len(exe_summary[data]) != 0:
            df_lat = pd.DataFrame(data=exe_summary['Latency'], columns=["Latency"])
            df_lat.index = range(1, len(df_lat.index)+1)

            all_df.append(df_lat)

        elif data == "IOPS" and len(exe_summary[data]) != 0:
            df_iops = pd.DataFrame(data=exe_summary['IOPS'], columns=["IOPS"])
            df_iops.index = range(1, len(df_iops.index)+1)

            all_df.append(df_iops)


    present_time = exe_summary["Date"]
    latest_version = exe_summary["Latest Version"].split("(")[0]
    baseline_version = exe_summary["Baseline Version"].split("(")[0]

    return all_df, latest_version, baseline_version, present_time

In [ ]:
exe_summary_df(exe_summary=exe_summary)

In [ ]:
for data in exe_summary:
    if len(exe_summary[data]) == 0:
        exe_summary[data] = ["No Regression Found"]

In [ ]:
exe_summary